In [1]:
import numpy as np
import scipy.linalg as sla

In [2]:
def decomLU(A):
    # com todos os números convertidos em double, se não converter pra double a divisão em (1) será truncada gerando bug
    U = np.array(A, dtype=np.float64) 
    n = U.shape[0]
    L = np.identity(n)
    P = np.identity(n)
    
    # decomposição (escalonamento)
    for c in range(n-1):
        # pivotamento parcial 
        row_i_for_permutation = np.abs(U[c:, c]).argmax() + c
        if np.abs(U[row_i_for_permutation, c]) > np.abs(U[c, c]):
            U[[c, row_i_for_permutation]] = U[[row_i_for_permutation, c]] # swap(c, row_i_for_permutation)
            #L[[c, row_i_for_permutation]] = L[[row_i_for_permutation, c]] # aqui a matriz de L altera
            P[[c, row_i_for_permutation]] = P[[row_i_for_permutation, c]] # aqui a matriz de Pivotamento altera
        # pivotamento parcial
        for l in range(c+1, n):
            """print(P)
            print(L)
            print(U)
            line,div,col = U[l],U[l,c]/U[c,c],U[c]
            print("U[l] - (U[l, c] / U[c, c]) * U[c]")
            print("U[{}] - (U[{}, {}] / U[{}, {}]) * U[{}]".format(l,l,c,c,c,c))
            print("{} - ({} / {}) * {}".format(line, U[l, c], U[c, c], col))
            print("{} - {} * {}".format(line, div, col))
            print("{} - {}".format(line, div * col))
            print("{}".format(line - div * col))"""
            if not (l+1 > n-1):
                ll=l+1
            else:
                ll = l % (n-1) + 1 + c
            L[ll, c] = (U[l, c] / U[c, c]) # (1)
            U[l] = U[l] - (U[l, c] * U[c]) / U[c, c]    
    # decomposição (escalonamento)
    P = np.transpose(P) # (3), a transposição de P está sendo feita para que P@L@U == A, 
    # coincidindo com o resultado de scipy.linalg.lu(A), sim scipy.linalg.lu(A) tranpoem P 
    return P,L,U

# método trivial para matriz triangular superior
def uppTriLinSys(A, b):
    n = len(b)
    x = np.empty(n)
    
    x[n - 1] = b[n - 1] / A[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = ( b[i] - np.sum( A[i, i + 1:] * x[i + 1:] ) ) / A[i, i]
            
    return x

def lowTriLinSys(A, b):
    n = m = len(b)
    x = np.empty(n)
    x[0] = b[0] / A[0, 0]
    
    for i in range(1, n):
        x[i] = ( b[i] - np.sum( A[i, :i] * x[:i] ) ) / A[i, i]
        
    return x

def solveViaLUdecomp(A, b):
    P,L,U = decomLU(A)
    P = np.transpose(P) # é necessáro destranpor a matriz para reverter a transposição em (3)
    y = lowTriLinSys(L, P@b)
    x = uppTriLinSys(U, y)
    return x

a)  
x1 + x2 + x3 = 1  
4x1 + 4x2 + 2x3 = 2  
2x1 + x2 − x3 = 0

In [3]:
A = np.array([[1,1,1], [4,4,2], [2,1,-1]]).astype(np.float64)
b = np.array([1,2,0]).astype(np.float64).astype(np.float64)

In [4]:
"decomposição do scipy - P,L,U", sla.lu(A), "decomposição decomLU(A) - P,L,U", decomLU(A) # P,L,U

('decomposição do scipy - P,L,U',
 (array([[0., 0., 1.],
         [1., 0., 0.],
         [0., 1., 0.]]),
  array([[ 1.  ,  0.  ,  0.  ],
         [ 0.5 ,  1.  ,  0.  ],
         [ 0.25, -0.  ,  1.  ]]),
  array([[ 4. ,  4. ,  2. ],
         [ 0. , -1. , -2. ],
         [ 0. ,  0. ,  0.5]])),
 'decomposição decomLU(A) - P,L,U',
 (array([[0., 0., 1.],
         [1., 0., 0.],
         [0., 1., 0.]]),
  array([[ 1.  ,  0.  ,  0.  ],
         [ 0.5 ,  1.  ,  0.  ],
         [ 0.25, -0.  ,  1.  ]]),
  array([[ 4. ,  4. ,  2. ],
         [ 0. , -1. , -2. ],
         [ 0. ,  0. ,  0.5]])))

In [5]:
P,L,U = decomLU(A)
plu = P@L@U
"A", A, "P@L@U", plu, "A == P@L@U", A == plu

('A',
 array([[ 1.,  1.,  1.],
        [ 4.,  4.,  2.],
        [ 2.,  1., -1.]]),
 'P@L@U',
 array([[ 1.,  1.,  1.],
        [ 4.,  4.,  2.],
        [ 2.,  1., -1.]]),
 'A == P@L@U',
 array([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [6]:
LU, piv = sla.lu_factor(A)
x1 = sla.lu_solve((LU,piv),b)
x2 = solveViaLUdecomp(A, b)
"sla.lu_solve()", x1, "solveViaLUdecomp()", x2, "sla.lu_solve() == solveViaLUdecomp()", x1 == x2

('sla.lu_solve()',
 array([ 1., -1.,  1.]),
 'solveViaLUdecomp()',
 array([ 1., -1.,  1.]),
 'sla.lu_solve() == solveViaLUdecomp()',
 array([ True,  True,  True]))

In [7]:
np.transpose(P)@A, L@U

(array([[ 4.,  4.,  2.],
        [ 2.,  1., -1.],
        [ 1.,  1.,  1.]]),
 array([[ 4.,  4.,  2.],
        [ 2.,  1., -1.],
        [ 1.,  1.,  1.]]))

In [8]:
P,L,U = decomLU(A)
y = lowTriLinSys(L, np.transpose(P)@b)
x = uppTriLinSys(U, y)
print("here", b, L@y)
print("here", y, U@x)
print("here", x)

here [1. 2. 0.] [2. 0. 1.]
here [ 2.  -1.   0.5] [ 2.  -1.   0.5]
here [ 1. -1.  1.]


b)  
7x1 − 7x2 + x3 = 1  
−3x1 + 3x2 + 2x3 = 2  
7x1 + 7x2 − 72x3 = 7

In [9]:
A = np.array([[7,-7,1], [-3,3,2], [7,7,-72]]).astype(np.float64)
b = np.array([1,2,7]).astype(np.float64)

In [10]:
"decomposição do scipy - P,L,U", sla.lu(A), "decomposição decomLU(A) - P,L,U", decomLU(A) # P,L,U

('decomposição do scipy - P,L,U',
 (array([[1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.]]),
  array([[ 1.        ,  0.        ,  0.        ],
         [ 1.        ,  1.        ,  0.        ],
         [-0.42857143,  0.        ,  1.        ]]),
  array([[  7.        ,  -7.        ,   1.        ],
         [  0.        ,  14.        , -73.        ],
         [  0.        ,   0.        ,   2.42857143]])),
 'decomposição decomLU(A) - P,L,U',
 (array([[1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.]]),
  array([[ 1.        ,  0.        ,  0.        ],
         [ 1.        ,  1.        ,  0.        ],
         [-0.42857143,  0.        ,  1.        ]]),
  array([[  7.        ,  -7.        ,   1.        ],
         [  0.        ,  14.        , -73.        ],
         [  0.        ,   0.        ,   2.42857143]])))

In [11]:
P,L,U = decomLU(A)
plu = P@L@U
"A", A, "P@L@U", plu, "A == P@L@U", A == plu

('A',
 array([[  7.,  -7.,   1.],
        [ -3.,   3.,   2.],
        [  7.,   7., -72.]]),
 'P@L@U',
 array([[  7.,  -7.,   1.],
        [ -3.,   3.,   2.],
        [  7.,   7., -72.]]),
 'A == P@L@U',
 array([[ True,  True,  True],
        [ True,  True, False],
        [ True,  True,  True]]))

In [ ]:
LU, piv = sla.lu_factor(A)
x1 = sla.lu_solve((LU,piv),b)
x2 = solveViaLUdecomp(A, b)
"sla.lu_solve()", x1, "solveViaLUdecomp()", x2, "sla.lu_solve() == solveViaLUdecomp()", x1 == x2

c)  
x1 + 2x2 + 3x3 + 4x4 = 20  
2x1 + 2x2 + 3x3 + 4x4 = 22  
3x1 + 3x2 + 3x3 + 4x4 = 22  
4x1 + 4x2 + 4x3 + 4x4 = 24

In [12]:
A = np.array([[1,2,3,4], [2,2,3,4], [3,3,3,4], [4,4,4,4]]).astype(np.float64)
b = np.array([20,22,22,24]).astype(np.float64)

In [13]:
"decomposição do scipy - P,L,U", sla.lu(A), "decomposição decomLU(A) - P,L,U", decomLU(A) # P,L,U

('decomposição do scipy - P,L,U',
 (array([[0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.]]),
  array([[1.  , 0.  , 0.  , 0.  ],
         [0.25, 1.  , 0.  , 0.  ],
         [0.5 , 0.  , 1.  , 0.  ],
         [0.75, 0.  , 0.  , 1.  ]]),
  array([[4., 4., 4., 4.],
         [0., 1., 2., 3.],
         [0., 0., 1., 2.],
         [0., 0., 0., 1.]])),
 'decomposição decomLU(A) - P,L,U',
 (array([[0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.]]),
  array([[1.  , 0.  , 0.  , 0.  ],
         [0.25, 1.  , 0.  , 0.  ],
         [0.5 , 0.  , 1.  , 0.  ],
         [0.75, 0.  , 0.  , 1.  ]]),
  array([[4., 4., 4., 4.],
         [0., 1., 2., 3.],
         [0., 0., 1., 2.],
         [0., 0., 0., 1.]])))

In [14]:
P,L,U = decomLU(A)
plu = P@L@U
"A", A, "P@L@U", plu, "A == P@L@U", A == plu

('A',
 array([[1., 2., 3., 4.],
        [2., 2., 3., 4.],
        [3., 3., 3., 4.],
        [4., 4., 4., 4.]]),
 'P@L@U',
 array([[1., 2., 3., 4.],
        [2., 2., 3., 4.],
        [3., 3., 3., 4.],
        [4., 4., 4., 4.]]),
 'A == P@L@U',
 array([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]]))

In [ ]:
LU, piv = sla.lu_factor(A)
x1 = sla.lu_solve((LU,piv),b)
x2 = solveViaLUdecomp(A, b)
"sla.lu_solve()", x1, "solveViaLUdecomp()", x2, "sla.lu_solve() == solveViaLUdecomp()", x1 == x2